In [1]:
!pip install pandas
!pip install opencv-python
!apt-get install -y libgl1
!pip install kaggle


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libbsd0 libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2
  libdrm-radeon1 libdrm2 libedit2 libgl1-amber-dri libgl1-mesa-dri
  libglapi-mesa libglvnd0 libglx-mesa0 libglx0 libllvm15 libmd0 libpciaccess0
  libsensors-config libsensors5 libx11-6 libx11-data libx11-xcb1 libxau6
  libxcb-dri2-0 libxcb-dri3-0 libxcb-glx0 libxcb-present0 libxcb-randr0
  libxcb-shm0 libxcb-sync1 libxcb-xfixes0 libxcb1 libxdmcp6 libxext6
  libxfixes3 libxshmfence1 libxxf86vm1
Suggested packages:
  pciutils lm-sensors
The following NEW packages will be installed:
  libbsd0 libdrm-amdgpu1 libdrm-common libdrm-intel

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import h5py
import shutil
import cv2
import random
import uuid

In [4]:

def add_noise(img):
    gauss_noise=np.zeros(img.shape,dtype=np.uint8)
    cv2.randn(gauss_noise,128,20)
    gauss_noise=(gauss_noise*0.5).astype(np.uint8)
    gn_img=cv2.add(img,gauss_noise)
    return gn_img

def random_rotate(img,low=-40,high=40):
    angle = np.random.randint(low,high)
    rows,cols = img.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle ,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    return dst

def vertical_flip(img):
    return cv2.flip(img, 1)

def horizontal_flip(img):
    return cv2.flip(img,0)

def horizontal_vertical_flip(img):
    return cv2.flip(img,-1)

def horizontal_shearing(img):
    M = np.float32([[1, np.random.uniform(), 0], [0, 1, 0], [0, 0, 1]])
    rows,cols = img.shape[:2]
    sheared_img = cv2.warpPerspective(img, M, (int(cols*1.5), int(rows*1.5)))
    return sheared_img

def vertical_shearing(img):
    rows,cols = img.shape[:2]
    M = np.float32([[1, 0, 0], [np.random.uniform(), 1, 0], [0, 0, 1]])
    sheared_img = cv2.warpPerspective(img, M,
                                 (int(cols*1.5),
                                 int(rows*1.5)))
    return sheared_img

def horizontal_vertical_shearing(img):
    rows, cols = img.shape[:2]
    M = np.float32([[1, np.random.uniform(), 0], [np.random.uniform(), 1, 0], [0, 0, 1]])
    sheared_img = cv2.warpPerspective(img, M,
                                 (int(cols*1.5),
                                 int(rows*1.5)))
    return sheared_img
def salt_and_pepper_noise(img):

    # Getting the dimensions of the image
    row , col = img.shape[:2]

    # Randomly pick some pixels in the
    # image for coloring them white
    # Pick a random number between 300 and 10000
    number_of_pixels = np.random.randint(50, 200)
    for i in range(number_of_pixels):

        # Pick a random y coordinate
        y_coord=np.random.randint(0, row - 1)

        # Pick a random x coordinate
        x_coord=np.random.randint(0, col - 1)

        # Color that pixel to white
        img[y_coord][x_coord] = 255

    # Randomly pick some pixels in
    # the image for coloring them black
    # Pick a random number between 300 and 10000
    number_of_pixels = np.random.randint(50,200)
    for i in range(number_of_pixels):

        # Pick a random y coordinate
        y_coord=np.random.randint(0, row - 1)

        # Pick a random x coordinate
        x_coord=np.random.randint(0, col - 1)

        # Color that pixel to black
        img[y_coord][x_coord] = 0

    return img

tasks = {
    "add_noise":add_noise,
    "random_rotate":random_rotate,
    "vertical_flip":vertical_flip,
    "horizontal_flip":horizontal_flip,
    "horizontal_vertical_flip":horizontal_vertical_flip,
    "horizontal_shearing":horizontal_shearing,
    "vertical_shearing":vertical_shearing,
    "horizontal_vertical_shearing":horizontal_vertical_shearing,
    "salt_and_pepper_noise":salt_and_pepper_noise
}

def data_augmentation(path):
    choices = list(tasks.keys())
    random_choice = random.choice(choices)
    img = cv2.imread(path)
    img = tasks[random_choice](img)
    return img



In [5]:
benign_files = os.listdir(os.path.join("new_training_data","benign"))

In [6]:
malignant_files = os.listdir(os.path.join("new_training_data","malignant"))

In [8]:
for _ in range(len(benign_files)-len(malignant_files)):
    file = random.choice(malignant_files)
    img = data_augmentation(os.path.join("new_training_data","malignant",file))
    filename = os.path.join("new_training_data","malignant",str(uuid.uuid4())+".jpg")
    cv2.imwrite(filename,img)